In [1]:
import pandas as pd
import os

In [9]:
DATA_DIR = os.getenv("DATA_DIR")
processed_network = os.path.join(DATA_DIR, "processed_network")
content_api = os.path.join(DATA_DIR, "content_api")

## Network data

In [2]:
edges_file_filtered = os.path.join(processed_network, "edges_graphsagetest_feb_01_18_doo_min15weight_wtext.csv.gz")
edges = pd.read_csv(edges_file_filtered, compression="gzip")

In [3]:
edges.head()

,src_node,dest_node,weight,src_cid,dest_cid,source,target
0,/government/publications/guidance-for-dependan...,/visa-fees,66,5ef7560d-7631-11e4-a3cb-005056011aef,1e333395-5dd5-4452-96a3-fbe939928761,33265,58314
1,/visa-fees,/find-a-visa-application-centre,1164,1e333395-5dd5-4452-96a3-fbe939928761,aa055bd6-fde1-4471-8ac2-ac86241b4898,58314,12343
2,/entering-staying-uk/family-visas,/uk-family-visa,377,d612c61e-22f4-4922-8bb2-b04b9202126e,305acc88-488e-44ec-8e5a-1a5de7819ba9,23720,25053
3,/uk-family-visa,/uk-family-visa/partner-spouse,26227,305acc88-488e-44ec-8e5a-1a5de7819ba9,305acc88-488e-44ec-8e5a-1a5de7819ba9,25053,14044
4,/uk-family-visa/partner-spouse,/government/publications/application-for-uk-vi...,2733,305acc88-488e-44ec-8e5a-1a5de7819ba9,5ef421be-7631-11e4-a3cb-005056011aef,14044,33797


In [5]:
edge_cids = set(list(edges.src_cid.values) + list(edges.dest_cid.values))

In [6]:
len(edge_cids)

31932

## Labelled and content_api extract

In [10]:
labelled_file = os.path.join(content_api, "labelled.csv.gz")
labelled = pd.read_csv(labelled_file, compression="gzip")

/Users/felisialoukou/.pyenv/versions/3.6.0/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
len(edge_cids.difference(set(labelled.content_id.values)))

14903

In [15]:
content_file = os.path.join(content_api, "merged_content_json.csv.gz")
content = pd.read_csv(content_file, compression="gzip")

In [16]:
len(edge_cids.difference(set(content.content_id.values)))

0

In [21]:
content_edges = content[content.content_id.isin(edge_cids)]\
                            [["content_id", 
                              "title",
                              "description",
                              "details"]].copy(deep=True)

In [22]:
print(content_edges.shape)
content_edges.drop_duplicates("content_id", inplace=True)
content_edges.shape

(31932, 4)

In [24]:
text_for_embedding = os.path.join(content_api, "text_for_embedding_reviewed.csv.gz")
content_edges.to_csv(text_for_embedding, compression="gzip", index=False)